In [1]:
# 필요한 라이브러리 설치 및 임포트
#!pip install pandas

import pandas as pd
import datetime

---

# [Step 2] 데이터 전처리

#### **<span style="color:blue">[2-0] air_21, air_22, weather_21, weather_22 데이터 로딩</span>**

In [2]:
# 데이터 로딩
air_21 = pd.read_csv("Air_21.csv")
air_22 = pd.read_csv("Air_22.csv")
weather_21 = pd.read_csv("Weather_21.csv")
weather_22 = pd.read_csv("Weather_22.csv")

In [3]:
air_21.tail(2)

,측정일시,SO2,CO,O3,NO2,PM10,PM25
8758,2021-12-31 22:00:00,0.003,0.4,0.029,0.015,20.0,9.0
8759,2021-12-31 23:00:00,0.003,0.4,0.026,0.016,20.0,9.0


In [4]:
weather_22.tail(2)

,측정일시,기온,강수량,풍속,습도,증기압,일조시간,일사량,적설량,구름분포량,최저운고,시정,지면온도,5cm지중온도
2158,2022-03-31 22:00:00,9.3,0.0,1.9,58,6.8,0.4,0.2,0.5,3,10.0,2000,7.3,11.4
2159,2022-03-31 23:00:00,8.4,0.0,1.2,61,6.7,0.4,0.2,0.5,1,10.0,2000,6.7,11.0


#### **<span style="color:blue">[2-1] air_21, air_22 의 '측정일시'를 활용하여 'time' 변수 생성</span>**

* air_21, air_22 의 '측정일시'를 활용하여 'time'변수 생성
* time 변수를 to_datetime으로 데이터 타입 변경
* 참고: 미세먼지 데이터는 1시-24시, 날씨 데이터는 0시-23시로 구성되어 있습니다. [2-5]에서 미세먼지와 날씨 데이터를 time 기준으로 합치려면 날씨 기준이 동일해야 합니다. 미세먼지 데이터에서 time 변수 생성 시 이를 미리 고려(예:측정일시 값 -1)하세요.

In [5]:
# 아래에 필요한 코드를 작성하고 결과를 확인합니다.
air_21['time'] = pd.to_datetime(air_21['측정일시'])
air_21 = air_21[['측정일시', 'time', 'SO2', 'CO', 'O3', 'NO2', 'PM10', 'PM25']]
air_21.head(2)

,측정일시,time,SO2,CO,O3,NO2,PM10,PM25
0,2021-01-01 00:00:00,2021-01-01 00:00:00,0.002,0.5,0.022,0.016,24.0,14.0
1,2021-01-01 01:00:00,2021-01-01 01:00:00,0.002,0.6,0.018,0.020,25.0,14.0


In [6]:
air_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   측정일시    8760 non-null   object        
 1   time    8760 non-null   datetime64[ns]
 2   SO2     8760 non-null   float64       
 3   CO      8760 non-null   float64       
 4   O3      8760 non-null   float64       
 5   NO2     8760 non-null   float64       
 6   PM10    8760 non-null   float64       
 7   PM25    8760 non-null   float64       
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 547.6+ KB


In [7]:
air_22['time'] = pd.to_datetime(air_22['측정일시'])
air_22 = air_22[['측정일시', 'time', 'SO2', 'CO', 'O3', 'NO2', 'PM10', 'PM25']]
air_22.head(2)

,측정일시,time,SO2,CO,O3,NO2,PM10,PM25
0,2022-01-01 00:00:00,2022-01-01 00:00:00,0.003,0.4,0.026,0.016,23.0,12.0
1,2022-01-01 01:00:00,2022-01-01 01:00:00,0.003,0.4,0.022,0.020,20.0,9.0


---

#### **<span style="color:blue">[2-2] weather_21, weather_22 의 '일시'를 활용하여 'time' 변수 생성</span>**

* weather_21, weather_22 의 '일시'를 활용하여 'time'변수 생성
* time 변수를 to_datetime으로 데이터 타입 변경

In [8]:
# 아래에 필요한 코드를 작성하고 결과를 확인합니다.
weather_21['time'] = pd.to_datetime(weather_21['측정일시'])
weather_21 = weather_21[['측정일시', 'time', '기온', '강수량', '풍속', '습도', '증기압', '일조시간', '일사량',
                         '적설량', '구름분포량', '최저운고', '시정', '지면온도', '5cm지중온도']]
weather_21.head(2)

,측정일시,time,기온,강수량,풍속,습도,증기압,일조시간,일사량,적설량,구름분포량,최저운고,시정,지면온도,5cm지중온도
0,2021-01-01 01:00:00,2021-01-01 01:00:00,-8.7,0.0,2.4,68,2.2,0.0,0.0,0.0,0,0.0,2000,-6.9,-1.0
1,2021-01-01 02:00:00,2021-01-01 02:00:00,-9.1,0.0,1.6,69,2.1,0.0,0.0,0.0,0,0.0,2000,-7.1,-1.1


In [9]:
weather_22['time'] = pd.to_datetime(weather_22['측정일시'])
weather_22 = weather_22[['측정일시', 'time', '기온', '강수량', '풍속', '습도', '증기압', '일조시간', '일사량',
                         '적설량', '구름분포량', '최저운고', '시정', '지면온도', '5cm지중온도']]
weather_22.head(2)

,측정일시,time,기온,강수량,풍속,습도,증기압,일조시간,일사량,적설량,구름분포량,최저운고,시정,지면온도,5cm지중온도
0,2022-01-01 00:00:00,2022-01-01 00:00:00,-8.5,0.0,1.9,41,1.3,0.0,0.0,0.0,0,0.0,2000,-7.0,-1.0
1,2022-01-01 01:00:00,2022-01-01 01:00:00,-9.2,0.0,1.8,42,1.3,0.0,0.0,0.0,0,0.0,2000,-7.2,-1.1


---

#### **<span style="color:blue">[2-3] 'time' 기준으로 데이터 합치기</span>**

* 미세먼지 데이터와 날씨 데이터를 'time' 기준으로 합쳐보세요.
* df_21에는 'time' 기준으로 21년도 미세먼지, 날씨 데이터를 합쳐보세요.
* df_22에는 'time' 기준으로 22년도 미세먼지, 날씨 데이터를 합쳐보세요.

In [10]:
# 아래에 필요한 코드를 작성하고 결과를 확인합니다.
air_21d = air_21.drop('측정일시', axis=1)
weather_21d = weather_21.drop('측정일시', axis=1)
df_21 = pd.merge(air_21d, weather_21d, how='inner', on='time')

In [11]:
df_21.head(2)

,time,SO2,CO,O3,NO2,PM10,PM25,기온,강수량,풍속,습도,증기압,일조시간,일사량,적설량,구름분포량,최저운고,시정,지면온도,5cm지중온도
0,2021-01-01 01:00:00,0.002,0.6,0.018,0.020,25.0,14.0,-8.7,0.0,2.4,68,2.2,0.0,0.0,0.0,0,0.0,2000,-6.9,-1.0
1,2021-01-01 02:00:00,0.002,0.6,0.013,0.025,27.0,16.0,-9.1,0.0,1.6,69,2.1,0.0,0.0,0.0,0,0.0,2000,-7.1,-1.1


In [12]:
air_22d = air_22.drop('측정일시', axis=1)
weather_22d = weather_22.drop('측정일시', axis=1)
df_22 = pd.merge(air_22d, weather_22d, how='inner', on='time')

In [13]:
df_22.head(2)

,time,SO2,CO,O3,NO2,PM10,PM25,기온,강수량,풍속,습도,증기압,일조시간,일사량,적설량,구름분포량,최저운고,시정,지면온도,5cm지중온도
0,2022-01-01 00:00:00,0.003,0.4,0.026,0.016,23.0,12.0,-8.5,0.0,1.9,41,1.3,0.0,0.0,0.0,0,0.0,2000,-7.0,-1.0
1,2022-01-01 01:00:00,0.003,0.4,0.022,0.020,20.0,9.0,-9.2,0.0,1.8,42,1.3,0.0,0.0,0.0,0,0.0,2000,-7.2,-1.1


---

#### **<span style="color:blue">[2-4] 사용하지 않을 변수 제거</span>**

* 머신러닝에 사용하지 않을 변수들을 제거해줍니다.
* df_21, df_22에 사용할 변수들만 넣어보세요.

In [14]:
# df_21, df_22에 사용할 변수들만 할당

In [15]:
# time 변수를 index로 세팅
df_21 = df_21.set_index('time')
df_22 = df_22.set_index('time')

In [16]:
df_21.tail(2)

,SO2,CO,O3,NO2,PM10,PM25,기온,강수량,풍속,습도,증기압,일조시간,일사량,적설량,구름분포량,최저운고,시정,지면온도,5cm지중온도
time,,,,,,,,,,,,,,,,,,,
2021-12-31 22:00:00,0.003,0.4,0.029,0.015,20.0,9.0,-7.5,0.0,3.0,37,1.3,0.0,0.02,0.1,0,10.0,2000,-6.2,-0.8
2021-12-31 23:00:00,0.003,0.4,0.026,0.016,20.0,9.0,-7.7,0.0,2.9,38,1.3,0.0,0.02,0.1,0,10.0,2000,-6.5,-0.9


---

#### **<span style="color:blue">[2-5] 변수들의 결측치 처리</span>**

In [17]:
# df_21, df_22의 결측치 확인
df_21.isna().sum()

SO2        0
CO         0
O3         0
NO2        0
PM10       0
PM25       0
기온         0
강수량        0
풍속         0
습도         0
증기압        0
일조시간       0
일사량        0
적설량        0
구름분포량      0
최저운고       0
시정         0
지면온도       0
5cm지중온도    0
dtype: int64

In [18]:
# df_21, df_22의 변수 중 '강수량(mm)'의 결측치를 처리



In [19]:
# df_21, df_22의 남은 결측치를 처리



In [20]:
# df_21, df_22의 결측치 재확인



---

#### **<span style="color:blue">[2-6] 전일 같은 시간 변수 추가</span>**

* 모델링에 유용한 변수로 전일 같은 시간(24시간 전) 미세먼지 농도 변수를 추가합니다. 
* 시계열 데이터 처리를 위한 shift 연산을 참고하세요.

In [21]:
# df_21, df_22의 index(time)를 month, day, hour 로 쪼개기 (year는 필요 없음)
#df_21['month'] = df_21.index.month
#df_21['day'] = df_21.index.day
#df_21['hour'] = df_21.index.hour
#df_21.head()

In [22]:
#df_22['hour'] = df_22.index.hour
#df_22.head()

In [35]:
# df_21, df_22에 전일 같은 시간 미세먼지 농도 변수(PM10_lag1) 추가
# 전일 같은 시간은 24시간 전 입니다.
df_21['PM10_lag1'] = df_21['PM10'].shift(24)
df_21[::24]

,SO2,CO,O3,NO2,PM10,PM25,기온,강수량,풍속,습도,...,일조시간,일사량,적설량,구름분포량,최저운고,시정,지면온도,5cm지중온도,PM10_1,PM10_lag1
time,,,,,,,,,,,,,,,,,,,,,
2021-01-01 01:00:00,0.002,0.6,0.018,0.020,25.0,14.0,-8.7,0.00000,2.4,68,...,0.00,0.000,0.000000,0,0.000000,2000,-6.9,-1.0,NaN,NaN
2021-01-02 01:00:00,0.003,0.4,0.031,0.010,25.0,9.0,-4.1,0.00000,2.3,39,...,0.00,0.010,0.000000,0,14.605263,2000,-4.7,-0.2,32.0,25.0
2021-01-03 01:00:00,0.003,0.4,0.030,0.008,52.0,12.0,-7.3,0.00000,2.4,48,...,0.05,0.020,0.000000,0,11.447368,2000,-6.9,-1.1,66.0,25.0
2021-01-04 01:00:00,0.003,0.6,0.008,0.034,32.0,18.0,-6.5,0.00000,1.3,49,...,0.00,0.015,0.000000,0,12.000000,2000,-7.3,-1.4,36.0,52.0
2021-01-05 01:00:00,0.002,0.5,0.013,0.026,39.0,24.0,-2.5,0.00000,0.5,75,...,0.00,0.020,0.000000,9,9.000000,1645,-1.6,0.0,47.0,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27 01:00:00,0.002,0.4,0.024,0.014,14.0,8.0,-12.4,0.02029,1.5,52,...,0.00,0.010,0.324880,0,12.166667,2000,-7.7,-1.0,20.0,15.0
2021-12-28 01:00:00,0.004,0.8,0.002,0.049,38.0,27.0,-6.8,0.00000,2.0,85,...,0.00,0.005,0.313397,0,8.153846,1779,-5.3,-0.6,43.0,14.0
2021-12-29 01:00:00,0.004,1.1,0.002,0.056,51.0,40.0,-3.5,0.00000,2.0,81,...,0.00,0.005,0.301914,0,13.000000,1108,-4.1,-0.2,55.0,38.0


---

#### **<span style="color:blue">[2-7] t+1 시점의 미세먼지 농도 데이터 생성</span>**

* t+1 시점은 1시간 후 입니다.
* t+1 시점의 미세먼지 농도 변수를 생성하세요.
* t+1 시점의 미세먼지 농도는 머신러닝 모델을 통해 예측하려는 y값(target) 입니다.

In [30]:
# df_21, df_22에 t+1 시점 변수(PM10_1) 추가
df_21['PM10_1'] = df_21['PM10'].shift(1)
df_22['PM10_1'] = df_22['PM10'].shift(1)
display(df_21.head())
display(df_22.head())

,SO2,CO,O3,NO2,PM10,PM25,기온,강수량,풍속,습도,증기압,일조시간,일사량,적설량,구름분포량,최저운고,시정,지면온도,5cm지중온도,PM10_1
time,,,,,,,,,,,,,,,,,,,,
2021-01-01 01:00:00,0.002,0.6,0.018,0.020,25.0,14.0,-8.7,0.0,2.4,68,2.2,0.0,0.0,0.0,0,0.0,2000,-6.9,-1.0,NaN
2021-01-01 02:00:00,0.002,0.6,0.013,0.025,27.0,16.0,-9.1,0.0,1.6,69,2.1,0.0,0.0,0.0,0,0.0,2000,-7.1,-1.1,25.0
2021-01-01 03:00:00,0.003,0.6,0.011,0.027,23.0,13.0,-9.3,0.0,1.1,70,2.1,0.0,0.0,0.0,0,0.0,2000,-7.3,-1.2,27.0
2021-01-01 04:00:00,0.003,0.6,0.008,0.032,24.0,14.0,-9.3,0.0,0.3,71,2.2,0.0,0.0,0.0,0,0.0,2000,-7.5,-1.3,23.0
2021-01-01 05:00:00,0.002,0.7,0.003,0.037,26.0,16.0,-9.7,0.0,1.9,72,2.1,0.0,0.0,0.0,0,0.0,2000,-7.6,-1.3,24.0


,SO2,CO,O3,NO2,PM10,PM25,기온,강수량,풍속,습도,증기압,일조시간,일사량,적설량,구름분포량,최저운고,시정,지면온도,5cm지중온도,PM10_1
time,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:00:00,0.003,0.4,0.026,0.016,23.0,12.0,-8.5,0.0,1.9,41,1.3,0.0,0.0,0.0,0,0.0,2000,-7.0,-1.0,NaN
2022-01-01 01:00:00,0.003,0.4,0.022,0.020,20.0,9.0,-9.2,0.0,1.8,42,1.3,0.0,0.0,0.0,0,0.0,2000,-7.2,-1.1,23.0
2022-01-01 02:00:00,0.003,0.5,0.014,0.028,20.0,9.0,-9.5,0.0,1.2,43,1.3,0.0,0.0,0.0,0,0.0,2000,-7.5,-1.3,20.0
2022-01-01 03:00:00,0.003,0.5,0.016,0.027,19.0,10.0,-9.3,0.0,1.4,46,1.4,0.0,0.0,0.0,0,0.0,2000,-7.6,-1.4,20.0
2022-01-01 04:00:00,0.003,0.5,0.005,0.040,24.0,11.0,-9.6,0.0,1.7,48,1.4,0.0,0.0,0.0,0,0.0,2000,-7.6,-1.4,19.0


In [32]:
df_21.shape, df_22.shape

((8759, 20), (2160, 20))

In [25]:
# 결측치가 있다면 처리
df_21['PM10_1'].fillna(

---

#### **<span style="color:blue">[2-8] train, test 데이터 분리</span>**

* 21년도 데이터(df_21)를 train 데이터로 저장하세요. y 값을 제외하고 train_x로 저장한 후 y 값은 train_y로 저장하세요.
* 22년도 데이터(df_22)를 test 데이터로 저장하세요. y 값을 제외하고 test_x로 저장한 후 y 값은 test_y로 저장하세요.
* 각각의 데이터프레임을 csv 파일로 저장하세요. (train_x.csv / train_y.csv / test_x.csv / test_y.csv)
* y값은 'PM10_1' 즉, t+1 시점의 미세먼지 농도입니다.

In [26]:
# 아래에 필요한 코드를 작성하고 결과를 확인합니다.



In [27]:
# 각각의 데이터프레임을 csv 파일로 저장 (train_x.csv / train_y.csv / test_x.csv / test_y.csv)

